# Perspective-Viewer Web Component

[Perspective](https://github.com/finos/perspective#readme) is an interactive visualization component for large, real-time datasets. It comes with the `perspective-viewer` web component.

It enables analysts and traders at large banks like J.P.Morgan to understand their data. But it could be usefull for analysts, engineers, scientists, data engineers and data scientists in general.

[Panel](https://panel.holoviz.org/) is a powerfull framework for creating awesome analytics apps in Python.

In this notebook we demonstrate how to use the `perspective-viewer` web component with Panel.

![Perspective](https://www.finos.org/hs-fs/hubfs/perspective.png?width=4256&name=perspective.png)

#### Author(s)

- [Marc Skov Madsen](https://datamodelanalytics.com), [awesome-panel.org](https://awesome-panel.org)

#### Tags

[#Perspective](https://github.com/finos/perspective#readme)
[Panel](https://panel.holoviz.org/)
[#Python](https://www.python.org/)

#### License

[MIT](https://opensource.org/licenses/MIT)

## Code

In [ ]:
import panel as pn
import param

from panel.pane.web_component import WebComponent
from bokeh.models import ColumnDataSource

JS_FILES = {
    "perspective": "https://unpkg.com/@finos/perspective@0.4.7",
    "perspective_viewer": "https://unpkg.com/@finos/perspective-viewer@0.4.7",
    "perspective_viewer_datagrid": "https://unpkg.com/@finos/perspective-viewer-datagrid@0.4.7/dist/umd/perspective-viewer-datagrid.js",
    "perspective_viewer_d3fc": "https://unpkg.com/@finos/perspective-viewer-d3fc@0.4.7",
    "perspective_viewer_hypergrid": "https://unpkg.com/@finos/perspective-viewer-hypergrid@0.4.7",
    # "perspective-jupyterlab": "https://unpkg.com/@finos/perspective-jupyterlab@0.4.7",
}

JS_FILES = {
    "perspective": "https://unpkg.com/@finos/perspective@0.4.7/dist/umd/perspective.js",
    "perspective_viewer": "https://unpkg.com/@finos/perspective-viewer@0.4.7/dist/umd/perspective-viewer.js",
    "perspective_viewer_datagrid": "https://unpkg.com/@finos/perspective-viewer-datagrid@0.4.7/dist/umd/perspective-viewer-datagrid.js",
    "perspective_viewer_hypergrid": "https://unpkg.com/@finos/perspective-viewer-hypergrid@0.4.7/dist/umd/perspective-viewer-hypergrid.js",
    "perspective_viewer_d3fc": "https://unpkg.com/@finos/perspective-viewer-d3fc@0.4.7/dist/umd/perspective-viewer-d3fc.js",
}

CSS_FILES = {
    "all": "https://unpkg.com/@finos/perspective-viewer@0.4.7/dist/umd/all-themes.css",
    "material": "https://unpkg.com/@finos/perspective-viewer@0.4.7/dist/umd/material.css",
    "material_dark": "https://unpkg.com/@finos/perspective-viewer@0.4.7/dist/umd/material.dark.css",
    "material_dense": "https://unpkg.com/@finos/perspective-viewer@0.4.7/dist/umd/material-dense.css",
    "material_dense_dark": "https://unpkg.com/@finos/perspective-viewer@0.4.7/dist/umd/material-dense.dark.css",
    "vaporwave": "https://unpkg.com/@finos/perspective-viewer@0.4.7/dist/umd/vaporwave.css",
}

THEMES = {
    "material": "perspective-viewer-material",
    "material-dark": "perspective-viewer-material-dark",
    "material-dense": "perspective-viewer-material-dense",
    "material-dense-dark": "perspective-viewer-material-dense-dark",
    "vaporwave": "perspective-viewer-vaporwave",
}
#Hack: When the user drags some of the columns, then the class attribute contains "dragging" also.
THEMES_DRAGGING = {key + " dragging": value + " dragging" for key, value in THEMES.items()}
THEMES = {**THEMES, **THEMES_DRAGGING}

from enum import Enum

# Source: https://github.com/finos/perspective/blob/e23988b4b933da6b90fd5767d059a33e70a2493e/python/perspective/perspective/core/plugin.py#L49
class Plugin(Enum):
    '''The plugins (grids/charts) available in Perspective.  Pass these into
    the `plugin` arg in `PerspectiveWidget` or `PerspectiveViewer`.
    Examples:
        >>> widget = PerspectiveWidget(data, plugin=Plugin.TREEMAP)
    '''
    HYPERGRID = 'hypergrid'  # hypergrid
    GRID = 'datagrid'  # hypergrid

    # YBAR = 'y_bar'  # highcharts
    # XBAR = 'x_bar'  # highcharts
    # YLINE = 'y_line'  # highcharts
    # YAREA = 'y_area'  # highcharts
    # YSCATTER = 'y_scatter'  # highcharts
    # XYLINE = 'xy_line'  # highcharts
    # XYSCATTER = 'xy_scatter'  # highcharts
    # TREEMAP = 'treemap'  # highcharts
    # SUNBURST = 'sunburst'  # highcharts
    #HEATMAP = 'heatmap'  # highcharts

    YBAR_D3 = 'd3_y_bar'  # d3fc
    XBAR_D3 = 'd3_x_bar'  # d3fc
    YLINE_D3 = 'd3_y_line'  # d3fc
    YAREA_D3 = 'd3_y_area'  # d3fc
    YSCATTER_D3 = 'd3_y_scatter'  # d3fc
    XYSCATTER_D3 = 'd3_xy_scatter'  # d3fc
    TREEMAP_D3 = 'd3_treemap'  # d3fc
    SUNBURST_D3 = 'd3_sunburst'  # d3fc
    HEATMAP_D3 = 'd3_heatmap'  # d3fc

    CANDLESTICK = 'd3_candlestick'  # d3fc
    CANDLESTICK_D3 = 'd3_candlestick'  # d3fc
    OHLC = 'd3_ohlc'  # d3fc
    OHLC_D3 = 'd3_ohlc'  # d3fc

    @staticmethod
    def options():
        return list(c.value for c in Plugin)

class PerspectiveViewer(WebComponent):
    html = param.String(
        """
    <perspective-viewer class='perspective-viewer-material-dark' style="height:100%;width:100%"></perspective-viewer>
    """
    )
    attributes_to_watch = param.Dict({
        "class": "theme",
        "plugin": "plugin",
        "rows": "rows",
        "row-pivots": "row_pivots",
        "columns": "columns",
        "column-pivots": "column_pivots",
        "sort": "sort",
        "aggregates": "aggregates", # Have not been able to manually test this one
        "filters": "filters",
    })

    theme = param.ObjectSelector("perspective-viewer-material-dark", objects=THEMES)
    plugin = param.ObjectSelector(Plugin.GRID.value, objects=Plugin.options())
    rows = param.List(None)
    row_pivots = param.List(None)
    column_pivots = param.List(None)
    columns = param.List(None)
    aggregates = param.List(None)
    sort = param.List(None)
    filters = param.List(None)

    data = param.DataFrame(doc="""The data will be reloaded in full when ever it changes.""")



    def __init__(self, **params):
        self.param.column_data_source_orient.default="records"
        self.param.column_data_source_load_function.default="load"

        super().__init__(**params)
        self._set_column_data_source()

    @param.depends("data", watch=True)
    def _set_column_data_source(self):
        if not self.data is None:
            self.column_data_source = ColumnDataSource(ColumnDataSource.from_df(self.data))
        else:
            self.column_data_source = ColumnDataSource()
    
    @staticmethod
    def get_imports():
        extension = ""
        for js in JS_FILES.values():
            extension += f'<script src="{js}"></script>'
        for css in CSS_FILES.values():
            extension += f'<link rel="stylesheet" href="{css}" is="custom-style">'
        
        return extension
        
    
    @staticmethod
    def config():
        """Add .js and .css files to configuration"""
        for key, value in JS_FILES.items():
            pn.config.js_files[key] = value
        pn.config.css_files.append(CSS_FILES["all"])


## Output

### Configuration

In [ ]:
# For some yet unknown reason this is needed in Notebook
from bokeh.resources import INLINE
from bokeh.io import output_notebook
output_notebook(resources=INLINE)

In [ ]:
PerspectiveViewer.config() 

# Loads .js and .css files in Notebook
pn.extension()

### Simple Example

In [ ]:
pn.config.sizing_mode = "stretch_width"

import pandas as pd
simple_data = [
    {"x": 1, "y": "a", "z": True},
    {"x": 2, "y": "b", "z": False},
    {"x": 3, "y": "c", "z": True},
    {"x": 4, "y": "d", "z": False},
]
simple_dataframe = pd.DataFrame(simple_data)
simple_perspective_viewer = PerspectiveViewer(height=200, data=simple_dataframe)
simple_perspective_viewer

### Perspective Panel App

In [ ]:
DARK_BACKGROUND="rgb(42, 44, 47)"
DARK_COLOR = "white"
PERSPECTIVE_LOGO = "https://perspective.finos.org/img/logo.png"
PANEL_LOGO="https://panel.holoviz.org/_static/logo_horizontal.png"
DATA = "PerspectiveViewerData.csv" # Source: https://datahub.io/core/s-and-p-500-companies-financials

dataframe = pd.read_csv(DATA)

perspective_viewer = PerspectiveViewer(height=600, data=dataframe)

top_app_bar = pn.Row(
    pn.pane.PNG(PERSPECTIVE_LOGO, height=50, margin=(10,25,10,10)),
    pn.pane.PNG(PANEL_LOGO, height=40, margin=(10,0,10,0)),
    pn.layout.HSpacer(),
)

settings_parameters = ['theme',
 'row_pivots',
 'plugin',
 'columns',
 'aggregates',
 'filters',
 'sort',
 'rows',
 'column_pivots']

settings_pane = pn.Param(
    perspective_viewer, 
    parameters=settings_parameters, 
    width=200, 
    sizing_mode="stretch_height",
    background="#9E9E9E",
)

app = pn.Column(
    top_app_bar,
    pn.Row(
        perspective_viewer,
        settings_pane,
    ),
    background=DARK_BACKGROUND
)
app

In [ ]:
def show(event):
    perspective_viewer.height=1000
    app.show()
    
button = pn.widgets.Button(name="DEPLOY TO SERVER", button_type="success")
button.on_click(show)
button

In [ ]:
perspective_viewer.height=600